In [3]:
# Data
import numpy as np
import pandas as pd

import time
from collections import Counter
import matplotlib.pyplot as plt  

# Ontology
#!pip install owlready2 
from owlready2 import *
import re  # To separate words based on capital letters in onto classes & to split search queries
#!pip install EMMOntoPy #Special EMMO package
# from ontopy import get_ontology


# GUI
import sys
#!pip install PyQt5
import PyQt5
from functools import partial # To connect signals with methods that need to take extra arguments
    # Import 'QApplication' and all the required widgets from PyQt.QtWidgets
from PyQt5.QtWidgets import QApplication
from PyQt5.QtWidgets import QMainWindow
from PyQt5.QtWidgets import QWidget
    #layout
from PyQt5.QtCore import Qt 
from PyQt5.QtWidgets import QGridLayout # Class to create objects for arranging buttons
from PyQt5.QtWidgets import QLineEdit   # Class to create and eddit buttons and dislpay
from PyQt5.QtWidgets import QPushButton # ?
from PyQt5.QtWidgets import QVBoxLayout # Class to create layout (display, buttons)
from PyQt5.QtWidgets import QListWidget # Class to create output screen
from PyQt5.QtWidgets import QLabel      # Title
from PyQt5.QtGui import QFont           # Font
# from PyQt5.QtGui import                 #? font
from PyQt5.QtGui import QPalette
from PyQt5.QtGui import QColor

print("Import done")
#import dash #(version .. ) pip install dash
#import dash_core_components as dcc
#import dash_html_components as html

Import done


## Ontology

In [4]:
"""Import Ontology & Select classes"""

# write the location to the ontology to the onto_path list
onto_path = ["https://raw.githubusercontent.com/SanliFaez/FAIR-Battery-knowledgebase/main/Ontology/FAIR-Battery-ontology.owl"]
    # Note that all the ontology files has to be owl files.
#onto_path = ["https://raw.githubusercontent.com/hendelhendel/FAIR_Battery/main/Ontology/flowtest.owl"]
#onto_path = ["https://github.com/hendelhendel/FAIR_Battery/blob/main/Ontology/flowbatterytest.owl"]
    
# Import ontology
try:
    onto = get_ontology(onto_path[0]).load()
except:
    pass 

onto = get_ontology(onto_path[0]).load()

# Collecting classes from ontology in a list
class_raw = list(get_ontology(onto_path[0]).load().classes())

# select classes by Prefix, suffix, nametags
tag = 'ElectrochemicalFlowCell'
prefix = 'electrochemistry.'

ClassCleaner = lambda x : re.sub('_',  ' ',\
                                 re.sub(r"(?<=\w)([A-Z])", r" \1", \
                                 str(x).removesuffix(tag).removeprefix(prefix))) \
                                    if (str(x).find(prefix) != -1) else "!EMPTY CLASS" 
                            

class_select = list(map(ClassCleaner, filter(lambda x : (str(x).find(tag) != -1), class_raw)))

print("Your imported ontology with "  + str(len(class_select)) + " classes is ready to use")


Your imported ontology with 33 classes is ready to use


## Process Data

In [5]:
"""Import Zotero Data Base from Github"""

# Import raw data from github repository as dataframe
data_path = 'https://raw.githubusercontent.com/SanliFaez/FAIR-Battery-knowledgebase/main/Datamanagement/Data_Raw.csv'
df_raw = pd.read_csv(data_path)


In [6]:
"""Data Process Functions"""

# Function Text Cleaner
f_CleanText = lambda text : re.split("\. |\! |\? ", text.lower()) if (type(text) == str)  else ["Not Available"]
    # Cleans text split scentices.
    # Output is of strings per for every text inputed. 
    # Example: Input = sentence1.sentence2!sentence3? --> Output = [sentence1,sentence2,sentence3]
    
    
# Function Keyword Search    
f_SearchKeyword = lambda text, keyword : text if (text.find(keyword) != -1) else None 
    # Searches text for keyword
    # Input: text = string or list of strings, keyword = string or list strings
    # Example: 
        # text = [sentence1,sentence2,sentence_A,sentence_B]
        # keyword = len(text)*[A]
        # Output  = [0,0,sentence_A,0]
        
# Function Scan lists of lists with text
f_ScanList = lambda text, keyword: list(map(f_SearchKeyword, text , len(text)*[keyword]))  
    # Example: 
        # text = [[sentence1,sentence2],[sentence_A,sentence_B]]
        # keyword = [A]
        # Output  = [[0,0],[sentence_A,0]]
                                        
#f_ScanList = lambda text, keyword: f_SearchKeyword(text, 'keyword') #if (type(text) == str) else "appel"
#f_ScanList = lambda text, keyword: len(text) if (type(text) == str) else "appel"

In [7]:
"""Process data"""

# Function Keyword Search    
f_SearchKeyword2 = lambda text, keyword : 'y' if (text.find(keyword) != -1) else 'n' 
    # Searches text for keyword
    # Input: text = string or list of strings, keyword = string or list strings
    # Example: 
        # text = [sentence1,sentence2,sentence_A,sentence_B]
        # keyword = len(text)*[A]
        # Output  = [n,n,y,n]
        # Function Scan lists of lists with text
f_ScanList2 = lambda text, keyword: list(map(f_SearchKeyword2, text , len(text)*[keyword]))  
    # Example: 
        # text = [[sentence1,sentence2],[sentence_A,sentence_B]]
        # keyword = [A]
        # Output  = [[n,n],[y,n]]
        
"""Process Data"""
start2 = time.time() # Measure time

# New data frame to store processed data
df_processed2 = df_raw.copy()
SourceText = "Abstract Note" #Column name text source
ProcessedText = "Clean Abstract Note" # Column name processed text

# clean abstract data 
df_processed2[ProcessedText] = df_raw[SourceText].map(f_CleanText)

# function to search data for ontology classes !! depends on df_processed !!
f_OntoSearch = lambda onto : list(map(f_ScanList2, df_processed2[ProcessedText], len(df_processed2[ProcessedText])*[onto.lower()]))

# Search data ontology class and store in df
def f_AddSearchResults(df, ontology):
    for onto in ontology: 
        df[str(onto)] = f_OntoSearch(onto)

f_AddSearchResults(df_processed2, class_select)

end2 = time.time() # Measure time
calc_time = end2-start2  # Measure time

print("Your " + str(len(df_processed2)) + " articles are searched on "  + str(len(class_select)) + " ontology classes in " + str(calc_time) + " seconds.")
print("type df_processed to view data frame")


Your 5644 articles are searched on 33 ontology classes in 0.3594551086425781 seconds.
type df_processed to view data frame


# Import Search Data

In [8]:
"""Import raw and processed data"""

df_processed = df_processed2

"""Get Columns names of processed data to build the interface
        Note: to search the processed data, it is necessary to discriminate between the article
            specific classes and the processed data classes"""
    
L_raw=list(df_raw.columns) # column names in raw data
L_processed=list(df_processed.columns) # column names in processed data set
# Type here below column names which are not onotlogy classes in the form ['name 2','name 2']
L_processed_non_onto=['Unnamed: 0','Clean Abstract Note','!EMPTY CLASS'] 

# Discriminate between column types
article_specs = L_raw # Article specifics such as: DOI, Pub year, Title, ...
ontology_classes = list((Counter(L_processed)-Counter(L_raw)-Counter(L_processed_non_onto)).elements()) # Ontology classes

"""Get text """
#method1
def get_text (text, code):
    #ans = ''
    l = []
    for i,char in enumerate(code):
        if char == 'y':
            l.append(text[i])
    #return ans
    return l

# #method2
# def get_text2 (df_text,df_code):
#     f_findtext = lambda df_texttext, df_codecode: df_texttext if (df_codecode=='y') else ''
#     f_txt = lambda df_text_row, df_code_row: list(map(f_findtext,df_text_row, df_code_row))
# #     f_txt = lambda df_text_row, df_code_row: [i for i in list(map(f_findtext,df_text_row, df_code_row)) if i is not None]
#     temp_ans = list(map(f_txt, df_text, df_code))
#     return temp_ans


for onto in ontology_classes:
    temp_funct = lambda df_text, df_code: get_text(df_text, df_code)
    ans = list(map(temp_funct,list(df_processed["Clean Abstract Note"]), list(df_processed[onto])))
    df_processed[onto] = ans

print("data colected")

data colected


# GUI

In [9]:
"""Create dictionaries of buttons for the interface"""

def define_buttons(btn_classes):
    storage = {}
    for clss in range(0,len(btn_classes.keys())): # Loop over button classes
        classname = list(btn_classes.keys())[clss]
        storage[classname] = {} # temporal storage per button class
        keys = btn_classes[list(btn_classes.keys())[clss]] # get list of buttons inside button class
        root = np.ceil(np.sqrt(len(keys))) # get the root of the number of classes for ordering the buttons
        #fill dict
        for key in range(len(keys)): # Loop over buttons in button class
            storage[classname][keys[key]] = np.divmod(key, root) #
    
    return storage

# Create a dict of button classes
windows = ["Search Assistance","Advanced Search","GO!"]
operations = ["and", "or", "-Clear Screen-","Search"]
results = ["Go To","Copy","Export to CSV"]
dict_button = {"Article Specs": article_specs, "Ontology Classes": ontology_classes, "Operations": operations, "Result": results, "Windows": windows}

# Collect all buttons and button classes in a dict of dict's
all_buttons = define_buttons(dict_button)

#all_buttons['Ontology Classes']
print("buttons ready")

buttons ready


In [10]:
#!/usr/bin/env python3

# Filename: pycalc.py

"""Rekenmachine is a simple calculator built using Python and PyQt5."""

ERROR_MSG = 'ERROR' # Handles the errors of a calcultation by the calculator


__version__ = '0.1'
__autor__ = 'Hendrik Snijder'

#  VIEW: Create a subclass of QMainWindow to setup the calculator's GUI
class PyCalcUi(QMainWindow):
    """VIEW of the rekenmachine(GUI). (public interface)"""
    
    # Properties
    def __init__(self):
        """View initializer."""
        super().__init__()
        # step3. Set some main window's properties
        self.setWindowTitle('Test de GUI')  #set the window title
        self.setFixedSize(1111, 731)  #set the size of the window
        
        # Set the central widget and the general layout (!keep same order!)
        self.generalLayout = QVBoxLayout() # USed to create display at top and buttons in a grid at bottom.
        self._centralWidget = QWidget(self) #Create a Qwidgit (central widget) = parent for rest of GUI
        self.setCentralWidget(self._centralWidget) 
        self._centralWidget.setLayout(self.generalLayout)
        # Create dislpayname
        #self.display.nameLabel.setText('Name:')
        
        #create the display and the buttons
        self._createDisplay() #creates display at the top of the window(method defined below)
        self._createClassButtons() #creates class buttons in a grid at the bottom (method defined below) hence e.g. self.buttons EXISTS!
        self._createLogicButtons() # Creates logical operator buttons
        self._createOutputScreen() # Creates outputscreen
      #  self._createNames()
        self._createResultButtons() #


 
    # Methods: Create View Elements
    def _createNames(self):
        """Create names"""
        self.nameLabel = QLabel(self)
        self.nameLabel.setText('Query')
        self.nameLabel.setFont(QFont("Query", 17, QFont.Bold))
     #   self.nameLabel.move(20, 29)
    
    def _createDisplay(self): 
        """Create the display."""
        # Create the display widget
        self.display = QLineEdit() # Create an object in class QLineEdit
        # Set some display's properties
        self.display.setFixedHeight(100) #fix display pixels
        self.display.setAlignment(Qt.AlignRight) #D isplay shows text as left alined
        self.display.setReadOnly(False) # Read-only to avoid direct editing in the display
        self.display.setFont(QFont("Query", 13, QFont.Bold))
        self.setDisplayText('Enter Query...')
  
        #Add the display to the general layout
        self.generalLayout.addWidget(self.display)
        
    def _createClassButtons(self):
        """Create Buttons"""
        self.buttons = {} #create empty dictionary to hold calculator buttons
        buttonsLayout = QGridLayout() # Create a temporary dictionary to store labels and positions (an object in QGridLayout)
        # Button text | position on the QGridLayout
        buttons = all_buttons['Ontology Classes'] # Call ontology class buttons 
                  
        # Create the buttons and add them to the grid layout in a for loop
        for btnText, pos in buttons.items():  # items is a function for dictionary (e.g. buttons)
            self.buttons[btnText] = QPushButton(btnText)
            self.buttons[btnText].setFixedSize(173,45)  #fixed size of button
            buttonsLayout.addWidget(self.buttons[btnText],pos[0], pos[1])
        # Add buttonsLayout to the general layout
        self.generalLayout.addLayout(buttonsLayout)

    def _createLogicButtons(self):
        """Create Buttons"""
        self.logicbuts = {} #create empty dictionary to hold calculator buttons
        logicbutsLayout = QGridLayout() # Create a temporary dictionary to store labels and positions (an object in QGridLayout)
        # Button text | position on the QGridLayout
        logicbuts = all_buttons['Operations'] # Call ontology class buttons 
                  
        # Create the buttons and add them to the grid layout in a for loop
        for Text, pos in logicbuts.items():  # items is a function for dictionary (e.g. buttons)
            self.logicbuts[Text] = QPushButton(Text)
            self.logicbuts[Text].setFixedSize(100,45)  #fixed size of button
            logicbutsLayout.addWidget(self.logicbuts[Text],pos[0], pos[1])
        # Add buttonsLayout to the general layout
        self.generalLayout.addLayout(logicbutsLayout)       
        
    def _createOutputScreen(self):
        """Create Output list"""
        # Create outputscreen widget
        self.outputscreen = QListWidget() # Create an object in de class QlistWidget
        # Set some output screen properties
        
        # Add outputscreen to the general layout
        self.generalLayout.addWidget(self.outputscreen)
        
        # Extra buttons in the OutputScreen
        #  gotobtn = QPushButton("Go To",self)
        #gotobtn.clicked.connect(self.o)
    
    def _createResultButtons(self):
        """Create Go To Button"""
        self.resultbuttons = {} # Create empty dictionary to hold result buttons
        resultbuttonsLayout = QGridLayout() # Create a temporary dictonary to store labels and positions 
        # result buttons test| position
        resultbuttons = all_buttons['Result'] # Call ontology class buttons 
        
        for text, pos in resultbuttons.items():
            self.resultbuttons[text] = QPushButton(text)
            self.resultbuttons[text].setFixedSize(100,45)
            resultbuttonsLayout.addWidget(self.resultbuttons[text],pos[0], pos[1])
        # Add resultbuttonsLayout to general layout
        self.generalLayout.addLayout(resultbuttonsLayout)
    
    
    # Methods: update Display View
    def setDisplayText(self, text):
        """Set display's text"""
#        text = text.replace('Enter Query...', '')
        self.display.setText(text) # Set and update the display's 'text'
        self.display.setFocus() # Set the cursor's focus on the display
        
    def displayText(self):
        """Get display's text"""
        return self.display.text() # Return display's current text (used as the math expr. to be evaluated.)
    
    def clearDisplay(self):
        """Clear the display"""
        self.setDisplayText('Enter Query...') # Set input screen back to 'Enter Query...' screen again keep it () to fully ')

         
    # Methods: Update outputscreen
    def setOutputScreenText(self, text): 
        """Set text outputscreen"""
        self.outputscreen.addItems(text)

    def clearOutputScreen(self):
        """Clear the outputscreen"""
        self.outputscreen.clear() #clears outputscreen 
        
        
# CONTROLLER: Create a Controller class to connect the GUI and the model
    # Controller class needs to perform ~ 3 tasks:
        # 1. acces the GUI public interface
        # 2. Handle the creation of math expressions
        # 3. Connect butten 'clicked' signals with the appropriate slots

class PyCalcCtrl:
    """PyCalc Controller class."""
    
    # Properties
    def __init__(self, model, view):
        """Controller initializer."""
        self._evaluate = model
        # Give an object of the vieuw PyCalcUi. 
            # Note: in main() view is defines as an object of PyCalcUi, hence all methods and properties of it applies!
            # Gives Full acces to the public interface
        self._view = view # imports view as a property
        # Connect signals and slots
        self._connectSignals()  # method defined below

        
    # Methods
    def _calculateResult(self):
        """Evaluate expressions."""
        # Functions to split and search query
        f_split_query = lambda query, separator : query.split(separator) # f_split_query(query = key1ANDkey2ORkey3) --> [[Key1,key2],[key3]]
        f_ANDsearch = lambda text, keyword1, keyword2 : "Ja" if (text.find(keyword1) != -1 and text.find(keyword2) != -1) else ''
        f_ORsearch = lambda text, keyword1, keyword2 : "Ja" if (text.find(keyword1) != -1 or text.find(keyword2) != -1) else ''
        
        # get data frame --> TODO: make this an input variable of _calculateResult
        df = df_processed # Data frame to search
        
        # ##########################
        # TO DO TO DO TO DO:
        # f_ANDsearch(df['keyword'],keyword,)
        
        
        
        # ##########################
        
        # Evaluate the expression entered in the display by the user ('=' is not displayed!)       
        que_screen = self._view.displayText()  # get input que from screen
        que =  list(map(f_split_query,f_split_query(que_screen, '| OR |'), len(f_split_query(que_screen, '| OR |'))*["| AND |"])) # split que by 'and' & 'or' into ontoclasses
        dictio = {} # store (overlapping) numbers together with search terms
        
        answer = 'Search Results: '# Create string to store result
#        count = [None] * len(parts) # storage to count number of times an onto class is found
        for parts in que:  #Searches for articles in wich all ontology classes of 'parts' are found.
            L_parts = []
            n = -1
            overlap_total = []
            name = ''
            for part in parts:
                name = name + part + ' & '
                L_part = []
                n = n +1
                for row in range(0,len(df[part])): # Search row in df where onto is found in text
                    if len(df[part][row]) != 0:
                        L_part.append(row)
                L_parts.append(L_part)
        
                overlap = lambda a,b: set(a) & set(b)
                if n == 0:
                    overlap_total = L_parts[0]
                else:
                    overlap_total = overlap(overlap_total,L_parts[n])
            dictio[str(parts)] = list(overlap_total)
            dictio[name] = list(overlap_total)
            
        
        # !NOTE: This loop can not be included in the previous loop!
        for parts in que:
            for row in dictio[str(parts)]:
                answer = answer + "\n   " + str(parts) + " found in the abstract of DOI:" + str(df["DOI"][row]) + "  in the sentence(s):"
                for onto in parts:
                    answer = answer + "\n           " + str(df[onto][row])
        
            
#    dictio[str(que[0])]     
#         for part in parts: # if part is not a class in the onto, an error arise!
# #           storage = {}  --> Here we can store the result per part, so that we can add the logical opporators to the answers
# #            storage.append()
#             for row in range(0,len(df)):  # Select rows to enter values of onto class
#                 try: # try if search term is in database
#                     if type(df[part].iloc[row]) == list: # Check if Onto class has a value (i.e. if it is not None)
#                         if type(df["DOI"].iloc[row]) == str and df[part].iloc[row][0] == 'yes':
#     #                        count = count + 1
#                             answer = answer + "\n   " + part + " found in the abstract of DOI:" + df["DOI"].iloc[row] + "  in the sentence(s)"
#                             for sentence in range(1,len(df[part].iloc[row])):
#                                 answer = answer + "\n           " + df[part].iloc[row][sentence] # store DOI of article found
#                 except KeyError: # if search term is not in database no search can be done
#                     answer = part + ' Not found in data base'

    
#         df = df_processed.copy()
#         que = [["Pump","Electrolyte","Membrane"],["Electrolyte"]] #testQue

#         # Get overlap: get the articles where a word or a set of words is found.
#         dictio = {} # store (overlapping) numbers together with search terms


    
    
        ###COUNtCOUNTCOUNTCOUNTCOUNT!!!
        # Add "ONTO class FOUNd X times in database" 
        # Add "class1 AND clss 2 found C times in data base" etc.
                    
        # Show anwser 
        result = answer
        # self._view.setDisplayText(result) # disabled to see input query on display
        self._view.setOutputScreenText(result.split("\n"))
    
    def _buildExpression(self, sub_exp):
        """Build expression.""" 
        # Clear display when error occurs
        if self._view.displayText() == ERROR_MSG:
            self._view.clearDisplay()
            
        # To handle the creation of expressions
        if sub_exp == 'and': 
            expression = self._view.displayText() + '| AND |'
            print('and')
        elif sub_exp == 'or':
            expression = self._view.displayText() + '| OR |'
        else: 
            expression = self._view.displayText() + sub_exp # expression to be evaluated + newest input
            expression = expression.replace('Enter Query...', '') # remove prefix Enter Query... after pressing a button
        self._view.setDisplayText(expression) # Uses function defined above, to display user input

    def _connectSignals(self):
        """Connect signals and slots."""
        # Connect the printable buttons with _buildExpression()
        for btnText, btn in self._view.buttons.items():  # for (a, b) in - buttons-list of (key, value) 
            if btnText not in {'-Clear Screen-','Search'}:
                btn.clicked.connect(partial(self._buildExpression, btnText))
        
        # Logic button AND 
    # self._view.logicbuts["and"].clicked.connect(partial(self._buidExpression, 'and'))
        
        # Logic buttons
        for logicbtnText, logicbtn in self._view.logicbuts.items():
            if logicbtnText not in {'-Clear Screen-','Search'}:
                logicbtn.clicked.connect(partial(self._buildExpression, logicbtnText))
        
        # connect 'search' button to calculate. check: (?) clicked.connect() is method of objects in PyCalcUi (?)
        self._view.logicbuts['Search'].clicked.connect(self._calculateResult) 
        # connect calculated result to display. returnPressed.connect() is method of objects in class QLineEdit
        self._view.display.returnPressed.connect(self._calculateResult) 
        # connect 'C' button to clear display
        self._view.logicbuts['-Clear Screen-'].clicked.connect(self._view.clearDisplay) 
        self._view.logicbuts['-Clear Screen-'].clicked.connect(self._view.clearOutputScreen)
        # connect Go To button

        
# MODEL: Create a function for the model to handle the calculator's operation
def evaluateExpression(expression):
    """Evaluate an expression."""
    try:
        # Use eval() to evaluate a string as an expression 
        result = str(eval(expression, {}, {})) # !IMPORTANT: only use eval() on trusted input due to security issues!!!
    except Exception: 
        result = ERROR_MSG
        
    return result
        

# Client code: Gives the actually routine!
def main(): # Defines the calculator's mainQ function (= best practise)
    """Main function."""
    # step 2. Create an instance of QApplication
    pycalc = QApplication(sys.argv) #creates object pycalc
    # Force the style to be the same on all OSs:
    pycalc.setStyle("Fusion")
    
    # Now use a palette to switch to dark colors:
    palette = QPalette()
    palette.setColor(QPalette.Window, QColor(53, 53, 53))
    palette.setColor(QPalette.WindowText, Qt.white)
    palette.setColor(QPalette.Base, QColor(25, 25, 25))
    palette.setColor(QPalette.AlternateBase, QColor(53, 53, 53))
    palette.setColor(QPalette.ToolTipBase, Qt.black)
    palette.setColor(QPalette.ToolTipText, Qt.white)
    palette.setColor(QPalette.Text, Qt.white)
    palette.setColor(QPalette.Button, QColor(53, 53, 53))
    palette.setColor(QPalette.ButtonText, Qt.white)
    palette.setColor(QPalette.BrightText, Qt.red)
    palette.setColor(QPalette.Link, QColor(42, 130, 218))
    palette.setColor(QPalette.Highlight, QColor(42, 130, 218))
    palette.setColor(QPalette.HighlightedText, Qt.black)
    pycalc.setPalette(palette)
    
    #step 4. Show the calculator's GUI
    view = PyCalcUi() # vieuw is object of class PyCalcUi
    view.show()  #shows the GUI
    # Create instances of the model and the controller
    model = evaluateExpression
    PyCalcCtrl(model=model, view=view)  # view is now also object of class PyCalcCtrl!
    #step5. Execute the calculator's main loop
    sys.exit(pycalc.exec_())  #Wruns application's event loop

if __name__ == '__main__': #This code will make sure that python execute the function main first. (See below)
    main()

#In this code, there is a function called main() that prints the phrase Hello World! when the Python interpreter executes it. 
#There is also a conditional (or if) statement that checks the value of __name__ and compares it to the string "__main__". 
#When the if statement evaluates to True, the Python interpreter executes main(). 
#You can read more about conditional statements in Conditional Statements in Python.

/tmp/ipykernel_1210606/1394657112.py:76: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  buttonsLayout.addWidget(self.buttons[btnText],pos[0], pos[1])
/tmp/ipykernel_1210606/1394657112.py:91: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  logicbutsLayout.addWidget(self.logicbuts[Text],pos[0], pos[1])
/tmp/ipykernel_1210606/1394657112.py:118: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  resultbuttonsLayout.addWidget(self.resultbuttons[text],pos[0], pos[1])


SystemExit: 0

/home/hendrik/anaconda3/envs/fairbattery/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
"""Data Search Functions"""
# --> NOTE: THESE FUNCTIONS GO INSIDE DEF OF CALC!! IN gui!!!
f_split_query = lambda query, separator : query.split(separator) # f_split_query(query = key1ANDkey2ORkey3) --> [[Key1,key2],[key3]]
#ans1 = f_split_query(que, '| OR |')
#ans2 = list(map(f_split_query,ans1, len(ans1)*["| AND |"]))

que = 'appel| AND |peer| OR |kiwi| OR |koek'#que = self._view.displayText()  # Input que
ans = list(map(f_split_query,f_split_query(que, '| OR |'), len(f_split_query(que, '| OR |'))*["| AND |"])) # split que by and & or
print(ans)

#SEARCH RESULTS IN .... SECONDS

#f_search = lambda splitted_query, article_text:  # f_search([[Key1,key2],[key3]],article_text) = [["key1 AND key2 found in: DOI sentice"],[]]

f_ANDsearch = lambda text, keyword1, keyword2 : "Ja" if (text.find(keyword1) != -1 and text.find(keyword2) != -1) else "Nee"
print(f_ANDsearch("Passen appels en peren in dezelfde zin?", "appels","peren"))
print(f_ANDsearch("Passen appels en peren in dezelfde zin?", "geenappels","peren"))
print(f_ANDsearch("Passen appels en peren in dezelfde zin?", "appels","geenperen"))
print(f_ANDsearch("Passen appels en peren in dezelfde zin?", "geenappels","geenperen"))

f_ORsearch = lambda text, keyword1, keyword2 : "Ja" if (text.find(keyword1) != -1 or text.find(keyword2) != -1) else "Nee"
print(f_ORsearch("Passen appels en peren in dezelfde zin?", "appels","peren"))
print(f_ORsearch("Passen appels en peren in dezelfde zin?", "geenappels","peren"))
print(f_ORsearch("Passen appels en peren in dezelfde zin?", "appels","geenperen"))
print(f_ORsearch("Passen appels en peren in dezelfde zin?", "geenappels","geenperen"))

boodschap = ["appels peren koeken","appels","koeken","sla, komkommer, kiwi's"]

#list(map(f_ANDsearch,boodschap,))

In [ ]:
"""Test wich method is faster"""
Lx=[-1,-2,-3,-4,-5,6,7,8,9,10]#*10000000
Ly=[1,2,2,3,3,3,4,4,4,4]#*10000000

#lambda's
functie1 = lambda x,y : x+y if (x<y) else x-y
functie2 = lambda x,y : x+y if (x>y) else x-y

s1 = time.time()
list(map(functie1,Lx,Ly)), list(map(functie2,Lx,Ly))
e1=time.time()

#definition
def functieverzameling(x,y):
    f1 =  lambda x,y : x+y if (x<y) else x-y
    f2 =  lambda x,y : x+y if (x>y) else x-y
    if type(x,y) == (list,list)
    return list(map(f1,x,y)), list(map(f2,x,y))

storage = 1000*[0]
x = 1000*[0]

for j in range(0,1000):
    x[j] = j
    s1 = time.time()
    list(map(functie1,Lx,Ly)), list(map(functie2,Lx,Ly))
    e1=time.time()
    t1 = e1-s1
    
    s2= time.time()    
    functieverzameling(Lx,Ly)
    e2= time.time()
    t2 = e2-s2
    storage[j] = t1-t2
    
fje = lambda lijst: 0 if (lijst < 0) else lijst
storage = list(map(fje,storage))
# Compute the x and y coordinates for points on a sine curve 
plt.title("which one is faster?") 
# Plot the points using matplotlib 
plt.plot(x,storage) 
plt.show() 


In [ ]:
type(3,3)

In [ ]:
"""---------------------------"""
"""---OLD CAN BE DELETED???---"""
"""---------------------------"""


"""Create dictionaries of buttons for the interface Method 1"""
s1=time.time()

def define_buttons(OntoClasses,ArticleClasses):
    
    # Buttons for ontology classes
    classbuttons = {} # create empty dict
    keys = OntoClasses # Ontology classes become keys
    root = np.ceil(np.sqrt(len(keys))) # number of rows and columns of the bottons
    # Fill dictionary
    for classnumber in range(len(keys)):
        classbuttons[keys[classnumber]] = np.divmod(classnumber, root)
    
    # Buttons for logic operations
    logicbuttons = {} # create empty dict
    logicbuttons["and"] = (0 , 0)
    logicbuttons["or"] = (0 , 1 )
    logicbuttons["-Clear Screen-"] = (0 , 2)
    logicbuttons["Search"] = (0, 3)
     
    # Buttons for article properties
    articlebuttons = {} # create empyt dict
    keys2 = ArticleClasses # Article specifics
    root2 = np.ceil(np.sqrt(len(keys2)))
    #fill dictionary
    for num in range(len(keys2)):
        articlebuttons[keys2[num]] = np.divmod(num, root2)
        
    return [classbuttons, articlebuttons, logicbuttons]

e1 = time.time()
All_buttons = define_buttons(article_specs,ontology_classes)


"""Create dictionaries of buttons for the interface Method 2"""
s2=time.time()

def define_buttons2(btn_classes):
    storage = len(btn_classes)*[None]
    for clss in range(len(btn_classes)):
        storage[clss] = {}
        keys = btn_classes[clss]
        root = np.ceil(np.sqrt(len(keys)))
        #fill dict
        for key in range(len(keys)):
            storage[clss][keys[key]] = np.divmod(key, root)
    
    return storage

operations = ["and", "or", "-Clear Screen-","Search"]
dict_button = {"Article Specs": article_specs, "Ontology Classes": ontology_classes, "Operations": operations}
ButtonClasses = [article_specs, ontology_classes,operations]
All_buttons2 = define_buttons2(ButtonClasses)

e2=time.time()
print(e2-s2)
#df_button = {"Article Specs": article_specs, "Ontology Classes": ontology_classes, "Operations": operations}
print(list(dict_button.keys()))
print(dict_button[list(dict_button.keys())[2]])
print(len(dict_button.keys()))
list(dict_button.keys())[1]

print(e1-s1)

In [ ]:
print(e1-s1)
print(e2-s2)
print(e3-s3)

In [ ]:
df = df_processed.copy()
que = [["Pump","Electrolyte","Membrane"],["Electrolyte"]] #testQue

# Get overlap: get the articles where a word or a set of words is found.
dictio = {} # store (overlapping) numbers together with search terms
for parts in que:
    L_parts = []
    n = -1
    overlap_total = []
    name = ''
    for part in parts:
        name = name + part + ' & '
        L_part = []
        n = n +1
        for row in range(0,len(df[part])): # Search row in df where onto is found in text
            if len(df[part][row]) != 0:
                L_part.append(row)
        L_parts.append(L_part)
        
        overlap = lambda a,b: set(a) & set(b)
        if n == 0:
            overlap_total = L_parts[0]
        else:
            overlap_total = overlap(overlap_total,L_parts[n])
    dictio[str(parts)] = list(overlap_total)
    dictio[name] = list(overlap_total)